1.   Initialize Parameters
2.   Classify and Predict
3.   Get Gradint Descent
4.   Update weight values
5.   Get loss function




<h1>NECESSARY LIBRARIES<h1>

In [1]:
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import nltk
nltk.download('stopwords')
from os import getcwd
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 

print('Import was successful')
# from utils import process_tweet, build_freqs

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Import was successful


<h1>1)DATA WRANGLING STAGE<h1>

In [6]:
data = pd.read_csv('Finalcleaneddataset.csv')

In [7]:
data.head()

,Unnamed: 0,date,id,content,username,polarity,sentiment
0,0,2021-05-23 12:05:20+00:00,1.39644E+18,anyth say islamophobia uk ðÿ ¤ ”,ZAJaffri,0.000000,2
1,2,2021-05-07 21:38:56+00:00,1.39078E+18,johnnydepp justiceforjohnnydepp appar up leve...,Stormember,-0.208333,0
2,3,2021-05-31 22:46:06+00:00,1.3995E+18,happi pride monthâ ™ ¥ ï ¸ � ðÿ � ³ï ¸ � â € ...,TsongaBoii,0.000000,2
3,4,2021-04-28 10:19:42+00:00,1.38735E+18,much hate ... hinduphobia,sumitisleo,-0.800000,0
4,5,2021-05-17 18:42:53+00:00,1.39436E+18,never condemn death baloch muslim pakhtoon mo...,singh27manish,0.000000,2


In [8]:
data['sentiment'].value_counts()

2    25825
1    12849
0     8020
Name: sentiment, dtype: int64

<h2>1a) removing the neutral sentiments<h2>

In [9]:
df = data[(data.sentiment==0) | (data.sentiment==1)]

<h2>1b) removing the missing values, extracting only "content" and "sentiment" columns<h2>

In [34]:
df.dropna(inplace=True)
df=df[['content', 'sentiment']]
df.reset_index(drop=True, inplace=True)
# df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,content,sentiment
0,johnnydepp justiceforjohnnydepp appar up leve...,0
1,much hate ... hinduphobia,0
2,coa mangel attack hamidmir plz tell hamidmir ...,1
3,muslim also face islamophobia jew israel terr...,1
4,tune onlyyouthenewremixesandmor 1hit mauricet...,1


<h2>1c) preparing data and performing train test split of 80-20<h2>

In [37]:
all_positive_tweets = df[df['sentiment'] == 1].to_numpy()
all_negative_tweets = df[df['sentiment'] == 0].to_numpy()

# print(all_positive_tweets[0:5],'\n\n',all_negative_tweets[0:5])
# print('class 0:', class_0.shape)
# print('class 1:', class_1.shape)

In [25]:
train_split_0 = int(class_0.shape[0]*0.8)
train_split_1 = int(class_1.shape[0]*0.8)

10276

In [46]:
train_pos = all_positive_tweets[:train_split_1]
test_pos = all_positive_tweets[train_split_1:]

train_neg = all_negative_tweets[:train_split_0]
test_neg = all_negative_tweets[train_split_0:]

train_x = np.vstack((train_pos, train_neg))  
test_x = np.vstack((test_pos,test_neg))

In [49]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

<h2>*Note:* additional code incase we need to undersample the give dataset<h2>

In [ ]:
# pos_tweets=[]
# neg_tweets=[]
# for i in range(len(x_rus)):
#   if x_rus[i][1] == 1:
#     pos_tweets.append(x_rus[i])
#   else:
#     neg_tweets.append(x_rus[i]) 

# print(len(pos_tweets))

55211


In [ ]:
# from imblearn.under_sampling import RandomUnderSampler
# from collections import Counter

# rus = RandomUnderSampler(random_state=42, replacement=True)# fit predictor and target variable
# x_rus, y_rus = rus.fit_resample(df, df.category)

# print('original dataset shape:', Counter(class_0))
# print('Resample dataset shape', Counter(y_rus))

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


original dataset shape: Counter({'clean_text': 1, 'category': 1})
Resample dataset shape Counter({0.0: 55211, 1.0: 55211})


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


<h2>*Note:* additional code incase some unfimiliar encoding is detected<h2>

In [ ]:
# import chardet # need to import this module to detect the encoding format


# for i in range(len(train_x)):
#   if isinstance(train_x[i][0], str) != True:
#     encode_type = chardet.detect(train_x[i][0])
#     print(encode_type)
#     Html = train_x[i][0].decode(encode_type['encoding'])
#   else:
#     pass
    

In [51]:
# # Print the shape train and test sets
# print("train_y.shape = " + str(train_y.shape))
# print("test_y.shape = " + str(test_y.shape))

train_y.shape = (16692, 1)
test_y.shape = (4174, 1)


<h2>1d) processing tweets i.e stemming and cleaning the content<h2>

In [74]:
def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet
    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    
    # encode_type = chardet.detect(tweet)
    # print(encode_type)
    # tweet = tweet.decode(encode_type['encoding'])
    
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'http.?://[^\s]+[\s]?', '',tweet)
    # tweet = re.sub("[^a-zA-Z]", " ",str(tweet))
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean


<h2>1e) creating frequency dictionary<h2>

In [77]:
def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    yslist = np.squeeze(ys).tolist()

    freqs = {}
    for y,tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs
    

In [75]:
content = []
for i in range(len(train_x)):
  content.append(train_x[i][0])

In [78]:
freqs = build_freqs(content, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 39022


In [141]:
# freqs

<h2>2)ML MODELS<h2>

<h2> 2a) Logistic Regression <h2>

<h3>SIGMOID FUNTION</h3>


In [80]:
def sigmoid(z): 
    '''
    Input:  z: is the input (can be a scalar or an array)
    Output: h: the sigmoid of z
    '''
    h = 1/(1 + np.exp(-z))
#     if h>=0.5:
#         h=1
#     else:
#         h=0    
    return h

<h3>COST FUNCTION AND GRADIENT</h3>

In [81]:
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    '''
    # get 'm', the number of rows in matrix x
    m = x.shape[0]
    
    for i in range(0, num_iters):
        
        # get z, the dot product of x and theta
        z = np.dot(x,theta)
        
        # get the sigmoid of z
        h = 1/(1 + np.exp(-z))
        
        # calculate the cost function
        J = (-1./m) * ( np.dot(np.transpose(y) , np.log(h)) + np.dot(np.transpose(1-y) , np.log(1-h)) )
        
        # update the weights theta
        theta = (theta) - (alpha/m) * np.dot(np.transpose(x) , (h-y))
        
    J = float(J)
    return J, theta

<h3>FEATURE EXTRACTION</h3>

In [82]:
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word, 1.0),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word, 0.0),0)
        
    assert(x.shape == (1, 3))
    return x


In [83]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(content), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(content[i], freqs)

# print(train_x_1)
# training labels corresponding to X
Y = train_y



In [84]:
# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 3000)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log


The cost after training is nan.
The resulting vector of weights is [2e-07, 0.0001726, -0.00024203]


In [85]:
print(J, theta)

nan [[ 1.98702915e-07]
 [ 1.72603404e-04]
 [-2.42031274e-04]]


<h3>PREDICT TWEETS</h3>

In [86]:
def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred = sigmoid(np.dot(x,theta))
    
    return y_pred

<h1>PERFORMANCE</h1>

In [88]:
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report



In [104]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    y_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1.0)
        else:
            # append 0 to the list
            y_hat.append(0.0)

    # accuracy = (y_hat==np.squeeze(test_y)).sum()/len(test_x)
    accuracy = metrics.accuracy_score(test_y,y_hat) 
    print("####################################\naccuracy:{0}\n####################################".format(accuracy))
    target_names = ['class 0', 'class 1']
    print(classification_report(test_y, y_hat,target_names=target_names))

    return y_hat



In [89]:
# my test content

test_x_1 = []
for i in range(len(test_x)):
  test_x_1.append(test_x[i][0])


In [133]:
y_hat = test_logistic_regression(test_x_1, test_y, freqs, theta)
# print(f"Logistic regression model's accuracy = {acc:.4f}")

####################################
accuracy:0.714183037853378
####################################
              precision    recall  f1-score   support

     class 0       0.69      0.47      0.56      1604
     class 1       0.72      0.87      0.79      2570

    accuracy                           0.71      4174
   macro avg       0.71      0.67      0.67      4174
weighted avg       0.71      0.71      0.70      4174



<h1>ERROR ANALYSIS</h1>

In [ ]:
print('Label Predicted Tweet')
for x,y in zip(test_x,test_y):
    y_hat = predict_tweet(test_x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('THE TWEET IS:', x)
        print('THE PROCESSED TWEET IS:', process_tweet(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

Label Predicted Tweet


TypeError: ignored

In [ ]:
# Feel free to change the tweet below
# my_tweet = 'This is a ridiculously bright movie. The plot was terrible and I was sad until the ending!'
my_tweet = 'I think rapists should be hanged because of the henious crimes they conducted against the humainty.'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

<h2>2b) Naive Bayes<h2>

In [135]:
def lookup(freqs, word, label):
    '''
    Input:
        freqs: a dictionary with the frequency of each pair (or tuple)
        word: the word to look up
        label: the label corresponding to the word
    Output:
        n: the number of times the word with its corresponding label appears.
    '''
    n = 0  # freqs.get((word, label), 0)

    pair = (word, label)
    if (pair in freqs):
        n = freqs[pair]

    return n

In [136]:
def train_naive_bayes(freqs, train_x, train_y):
    '''
    Input:
        freqs: dictionary from (word, label) to how often the word appears
        train_x: a list of tweets
        train_y: a list of labels correponding to the tweets (0,1)
    Output:
        logprior: the log prior. (equation 3 above)
        loglikelihood: the log likelihood of you Naive bayes equation. (equation 6 above)
    '''
    loglikelihood = {}
    logprior = 0

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # calculate V, the number of unique words in the vocabulary
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # calculate N_pos and N_neg
    N_pos = N_neg = 0
    for pair in freqs.keys():
        # if the label is positive (greater than zero)
        if pair[1] > 0:

            # Increment the number of positive words by the count for this (word, label) pair
            N_pos += freqs[pair]

        # else, the label is negative
        else:

            # increment the number of negative words by the count for this (word,label) pair
            N_neg += freqs[pair]

    # Calculate D, the number of documents
    D = len(train_y)

    # Calculate D_pos, the number of positive documents (*hint: use sum(<np_array>))
    D_pos = (len(list(filter(lambda x: x > 0, train_y))))

    # Calculate D_neg, the number of negative documents (*hint: compute using D and D_pos)
    D_neg = (len(list(filter(lambda x: x <= 0, train_y))))

    # Calculate logprior
    logprior = np.log(D_pos) - np.log(D_neg)

    # For each word in the vocabulary...
    for word in vocab:
        # get the positive and negative frequency of the word
        freq_pos = lookup(freqs,word,1)
        freq_neg = lookup(freqs,word,0)

        # calculate the probability that each word is positive, and negative
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)


    return logprior, loglikelihood

In [137]:
logprior, loglikelihood = train_naive_bayes(freqs, content, train_y)
print(logprior)
print(len(loglikelihood))

0.4710162086834231
30728


In [138]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
    Input:
        tweet: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Output:
        p: the sum of all the logliklihoods of each word in the tweet (if found in the dictionary) + logprior (a number)

    '''
    word_l = process_tweet(tweet)

    # initialize probability to zero
    p = 0

    # add the logprior
    p += logprior

    for word in word_l:

        # check if the word exists in the loglikelihood dictionary
        if word in loglikelihood:
            # add the log likelihood of that word to the probability
            p += loglikelihood[word]

    return p

In [139]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    Input:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of tweets classified correctly)/(total # of tweets)
    """
    accuracy = 0  # return this properly

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    # error = np.mean(np.absolute(y_hats-test_y))

    # Accuracy is 1 minus the error
    accuracy = metrics.accuracy_score(test_y,y_hats)

    ### END CODE HERE ###
    target_names = ['class 0', 'class 1']
    print(classification_report(test_y, y_hats,target_names=target_names))

    return accuracy


In [140]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x_1, test_y, logprior, loglikelihood)))

              precision    recall  f1-score   support

     class 0       0.80      0.81      0.81      1604
     class 1       0.88      0.87      0.88      2570

    accuracy                           0.85      4174
   macro avg       0.84      0.84      0.84      4174
weighted avg       0.85      0.85      0.85      4174

Naive Bayes accuracy = 0.8491
